<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Residual_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10
batch_size = 32

### Residual Unit

In [0]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')

    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h

### Residual Layer

In [0]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = list()
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

### Model

In [0]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, 3, 1, padding='same', activation='relu') # 28x28x8

        self.res1 = ResnetLayer(8, (16, 16), 3) # 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D(2) # 14x14x16

        self.res2 = ResnetLayer(16, (32, 32), 3) # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D(2) # 7x7x32

        self.res3 = ResnetLayer(32, (64, 64), 3) # 7x7x64

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)

        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

### Define train, test loop

In [0]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

### Load data

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### Define model, loss, optimizer and metric 

In [0]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [33]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.14608944952487946, Accuracy: 96.02833557128906, Test Loss: 0.06071119382977486, Test Accuracy: 98.04000091552734
Epoch 2, Loss: 0.07041860371828079, Accuracy: 98.06999969482422, Test Loss: 0.05759943649172783, Test Accuracy: 98.48999786376953
Epoch 3, Loss: 0.05099179595708847, Accuracy: 98.55999755859375, Test Loss: 0.041984349489212036, Test Accuracy: 98.8499984741211
Epoch 4, Loss: 0.042947400361299515, Accuracy: 98.711669921875, Test Loss: 0.03617538511753082, Test Accuracy: 98.97999572753906
Epoch 5, Loss: 0.034585271030664444, Accuracy: 99.01000213623047, Test Loss: 0.04555748403072357, Test Accuracy: 98.7699966430664
Epoch 6, Loss: 0.029480108991265297, Accuracy: 99.12999725341797, Test Loss: 0.053343262523412704, Test Accuracy: 98.40999603271484
Epoch 7, Loss: 0.025504417717456818, Accuracy: 99.24500274658203, Test Loss: 0.027010034769773483, Test Accuracy: 99.30999755859375
Epoch 8, Loss: 0.027176138013601303, Accuracy: 99.25, Test Loss: 0.04451655596494675, T

In [34]:
model.summary()

Model: "res_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           multiple                  80        
_________________________________________________________________
resnet_layer_9 (ResnetLayer) multiple                  8496      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
resnet_layer_10 (ResnetLayer multiple                  33376     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
resnet_layer_11 (ResnetLayer multiple                  132288    
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0 